In [1]:
### Finding a Protein Motif ### 

# given: Protein IDs 
# return: Protein ID
    # list of motif locations 

# goal: searching sequence for motif locations 

# create dataset from Protein IDs
    # create dictionary with protein sequences  
    # download fasta sequences from Uniprot with "http://www.uniprot.org/uniprot/uniprot_id.fasta"
    # add sequence to according ProteinID in dictionary 

# finding Motifs
    # -> loop through sequence 
    # -> compare AAs to first AA of Motif 
        # if equal compare second AA to second AA of motif -> repeat for all AAs of Motif 
        # if correct save location(s)  and protein ID in dictionary 
    # -> repeat for next sequence 
    # -> print dictionary
    
# output 
    # print Protein ID and list of locations

In [2]:
import urllib.request

### use text file from url as input ### 

### create list of protein IDs from Input ### 

ID_input = input(str)

ID_input = ID_input.split()
    
print(ID_input)

### loop through IDs and find sequences with url "http://www.uniprot.org/uniprot/uniprot_id.fasta" ### 

sequencedict = {} 

for item in range(len(ID_input)): 
    url = 'http://www.uniprot.org/uniprot/' + ID_input[item] + '.fasta'
    print(url)
    file = urllib.request.urlopen(url)
    
    fastalist = []
    
    ### decode url file and create list of decoded lines ### 
    
    for line in file:
        decoded_line = line.decode("utf-8")
        fastalist.append(decoded_line)
    
    ### create dictionary entry and join decoded lines to coherent string ### 
    
    ID = ID_input[item]

    sequencedict[ID] = list(fastalist[1:len(fastalist)])
    
    sequencedict[ID] = ''.join(fastalist[1:len(fastalist)]).replace("\n", '')



<class 'str'>P01044_KNH1_BOVIN
['P01044_KNH1_BOVIN']
http://www.uniprot.org/uniprot/P01044_KNH1_BOVIN.fasta


In [3]:
### finding motif in example Data ###

# data arrives in dictionary with Protein ID:Sequence 

### Create motif variable ### 

motif = "N{P}[ST]{P}"

### create list of Motif-AAs ### 

motif = list(motif)

### define dictionary for output ### 

outputdict = {}

### function that checks whether a whole motif has been found  ###

def motifcheck(motifcount, motif):
    if motifcount == len(motif):
        return True 

### add position to of found motif to output dictionary ### 

def addposition(n, outputdict, ID):
    outputdict[ID].append(n - 2)
    return outputdict

### create variable so loop knows which character in motif to look for ### 

motifcount = 0
matchcount = 0 

for item in range(len(ID_input)):
    
    ID = ID_input[item]
    sequence = sequencedict[ID]

    ### remove brakes from string by using string.replace and seperate characters with list() ###

    sequence = list(sequence.replace("\n", ''))

### loop through sequence + find motif positions + store them in output-dictionary ### 
    
    outputdict[ID] = []
    n = 0 
    
    while  n <= len(sequence):
        
        n += 1
        try: 
            sequence[n]
        except:
            break

        ### compare AA to AA in motif ###
        if sequence[n] != motif[motifcount]:
            
            ### identifying special characters ### 

            ### exception 1: any AA but not {...} ### 

            if motif[motifcount] == '{': 
                if sequence[n] != motif[motifcount + 1]:
                    motifcount += 3 
                    matchcount += 1
                    if motifcheck(motifcount, motif):
                        addposition(n, outputdict, ID)
                        motifcount = 0
                        n -= (matchcount - 1)
                        matchcount = 0 
                        continue
                    else: 
                        continue
                else:
                    n -= (matchcount)
                    matchcount = 0
                    motifcount = 0 
                    continue 

            ### Exception 2: any AA within [...] ### 

            elif motif[motifcount] == '[': 

                ### find number of strings in brackets ### 
                bracketlist = []

                for number in range(motifcount + 1, len(motif)): 
                    if  motif[number] != ']':
                        bracketlist.append(motif[number])
                        continue
                    else: 
                        break

                if sequence[n] in bracketlist: 
                    matchcount += 1 
                    motifcount += 4
                    continue 

                else: 
                    n -= (matchcount)
                    matchcount = 0
                    motifcount = 0 
                    continue
                    
            ### reset motifcount if AAs are not the same ### 

            else: 
                n -= (matchcount) 
                matchcount = 0
                motifcount = 0 
                continue 
        else:
            matchcount += 1 
            motifcount += 1

     ### reset motifcount if full motif was found and add Protein ID + position to output-dictionary ### 

            if motifcheck(motifcount, motif):
                    addposition(n, outputdict, ID)
                    motifcount = 0 
                    n -= (matchcount)
                    matchcount = 0
                    continue

    # -> print dictionary
print(outputdict)

{'P01044_KNH1_BOVIN': [47, 87, 168, 169, 197, 204]}


In [4]:
### print output in readable form ### 

for item in outputdict:
    
    if bool(outputdict[item]) != False: 
        print(item, str(outputdict[item]).replace('[', '').replace(',','').replace(']', ''), sep = '\n')
        


P01044_KNH1_BOVIN
47 87 168 169 197 204
